<a href="https://colab.research.google.com/github/Tomyao/CSE258/blob/master/Homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gzip
from collections import defaultdict
from google.colab import files

In [0]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [0]:
### Upload files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

<IPython.core.display.Javascript object>

User uploaded file "train.json.gz" with length 28944392 bytes
User uploaded file "pairs_Rating.txt" with length 440029 bytes
User uploaded file "pairs_Category.txt" with length 440027 bytes
User uploaded file "pairs_Visit.txt" with length 880029 bytes
User uploaded file "test_Category.json.gz" with length 2456035 bytes


In [0]:
### Save files

for fn in uploaded.keys():
  newFile = open(fn, "wb")
  newFileByteArray = bytearray(uploaded[fn])
  newFile.write(newFileByteArray)

In [0]:
### Check files are in directory

from os import listdir
from os.path import isfile, join
mypath = "./"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and not f.startswith('.')]
for myfiles in onlyfiles:
  print (myfiles)

pairs_Category.txt
train.json.gz
pairs_Visit.txt
example.txt
predictions_Rating.txt
randomData.txt
predictions_Visit.txt
pairs_Rating.txt
test_Category.json.gz


In [0]:
### Read in data and split into training and validation sets

allData = []

for l in readGz("train.json.gz"):
  allData.append(l)

allData_training = allData[:len(allData)/2]
allData_validation = allData[len(allData)/2:]

In [0]:
### Strip out unneccessary data for visit prediction in HW 3

# used for generating random non-visited pairs
visit_all = [{'userID' : d['userID'], 'businessID' : d['businessID']} for d in allData]

# has an extra field of categories for problem 3
visit_training = [{'userID' : d['userID'], 'businessID' : d['businessID'], 'categories' : d['categories']} for d in allData_training]

# does not have the extra field of categories since category tags are not included in test data
visit_validation = [{'userID' : d['userID'], 'businessID' : d['businessID']} for d in allData_validation]

In [0]:
### Randomly generate 100000 user/business pairs that weren't visited

#import random

# get all unique user and business IDs
#temp = set(d['userID'] for d in visit_all)
#unique_userID = []
#for val in temp:
#  unique_userID.append(val)
#temp = set(d['businessID'] for d in visit_all)
#unique_businessID = []
#for val in temp:
#  unique_businessID.append(val)

# start generation
#randomData = []

#totalGenerated = 0

#user_lastindex = len(unique_userID)-1
#business_lastindex = len(unique_businessID)-1

#while totalGenerated < 100000:
  # get the two IDs
#  userID = unique_userID[random.randint(0, user_lastindex)]
#  businessID = unique_businessID[random.randint(0, business_lastindex)]
  
  # check the two IDs
#  if ({'userID' : userID, 'businessID' : businessID} not in visit_all):
#    if ({'userID' : userID, 'businessID' : businessID} not in randomData):
#      randomData.append({'userID' : userID, 'businessID' : businessID})
#      totalGenerated += 1
      
      # Track progress
#      if totalGenerated % 5000 == 0:
#        print ("Progress: " + str(100 * (totalGenerated/100000.0)) + "% done")

### Randomly generating 100000 pairs takes a long time, so I just saved the
### results after one run into a file and load from it

randomData = []
for l in open('randomData.txt'):
  randomData.append(eval(l))

In [0]:
### Would-visit baseline: just rank which businesses are popular and which are not, and return '1' if a business is among the top-ranked

businessCount = defaultdict(int)
totalPurchases = 0

for l in visit_training:
  user,business = l['userID'],l['businessID']
  businessCount[business] += 1
  totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases/2: break


# testing on validation sets
correct = 0.0

# positive validation set
for data in visit_validation:
  if data['businessID'] in return1:
    correct += 1

# negative validation set
for data in randomData:
  if data['businessID'] not in return1:
    correct += 1

performance = correct/(len(visit_validation) + len(randomData))
print performance

0.62694


In [0]:
def getperf(percentile):
  businessCount = defaultdict(int)
  totalPurchases = 0

  for l in visit_training:
    user,business = l['userID'],l['businessID']
    businessCount[business] += 1
    totalPurchases += 1

  mostPopular = [(businessCount[x], x) for x in businessCount]
  mostPopular.sort()
  mostPopular.reverse()

  return1 = set()
  count = 0
  for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases * percentile: break


  # testing on validation sets
  correct = 0.0

  # positive validation set
  for data in visit_validation:
    if data['businessID'] in return1:
      correct += 1

  # negative validation set
  for data in randomData:
    if data['businessID'] not in return1:
      correct += 1

  performance = correct/(len(visit_validation) + len(randomData))
  print "Performance at " + str(percentile) + " percentile: " + str(performance)
  
for i in range (1,100):
  getperf(i/100.0)

Performance at 0.01 percentile: 0.50443
Performance at 0.02 percentile: 0.508845
Performance at 0.03 percentile: 0.51294
Performance at 0.04 percentile: 0.51673
Performance at 0.05 percentile: 0.520265
Performance at 0.06 percentile: 0.524175
Performance at 0.07 percentile: 0.527965
Performance at 0.08 percentile: 0.53207
Performance at 0.09 percentile: 0.53582
Performance at 0.1 percentile: 0.539515
Performance at 0.11 percentile: 0.54323
Performance at 0.12 percentile: 0.54704
Performance at 0.13 percentile: 0.55027
Performance at 0.14 percentile: 0.553945
Performance at 0.15 percentile: 0.557215
Performance at 0.16 percentile: 0.560435
Performance at 0.17 percentile: 0.563375
Performance at 0.18 percentile: 0.56639
Performance at 0.19 percentile: 0.56897
Performance at 0.2 percentile: 0.571735
Performance at 0.21 percentile: 0.57478
Performance at 0.22 percentile: 0.577425
Performance at 0.23 percentile: 0.580145
Performance at 0.24 percentile: 0.582445
Performance at 0.25 percentil

In [0]:
### Would-visit based on matching category

# get all unique users and businesses in training set
temp = set(d['userID'] for d in visit_training)
unique_userID = {}
for val in temp:
  unique_userID[val] = []
temp = set(d['businessID'] for d in visit_training)
unique_businessID = {}
for val in temp:
  unique_businessID[val] = []

# get categories for all unique users and businesses in training set
for data in visit_training:
  myuserID = data['userID']
  mybusinessID = data['businessID']
  mycategories = data['categories']
  for mycategory in mycategories:
    if mycategory not in unique_userID[myuserID]:
      unique_userID[myuserID].append(mycategory)
    if mycategory not in unique_businessID[mybusinessID]:
      unique_businessID[mybusinessID].append(mycategory)

In [0]:
predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  # get the two lists of categories
  if (u in unique_userID and i in unique_businessID):
    cat1 = unique_userID[u]
    cat2 = unique_businessID[i]
    matches = [x for x in cat1 if x in cat2]
    if len(matches) > 0:
      predictions.write(u + '-' + i + ",1\n")
    else:
      predictions.write(u + '-' + i + ",0\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

files.download('predictions_Visit.txt')

In [0]:
### Strip out unneccessary data for rating prediction in HW 3

rating_training = [{'userID' : d['userID'], 'businessID' : d['businessID'], 'rating' : d['rating']} for d in allData_training]
rating_validation = [{'userID' : d['userID'], 'businessID' : d['businessID'], 'rating' : d['rating']} for d in allData_validation]

In [0]:
### Define a performance measure

# Using MSE

def performance(predicted_ratings, actual_ratings):
  mse = 0.0
  for i in range(0, len(predicted_ratings)):
    mse += (predicted_ratings[i] - actual_ratings[i])**2
  return (mse/len(predicted_ratings))

In [0]:
### Trivial predictor: just return the global average
allRatings = []

for data in rating_training:
  allRatings.append(data['rating'])
  
globalAverage = sum(allRatings) / len(allRatings)
print ('Global average: ' + str(globalAverage))

predictions = []
for i in range(0,len(rating_validation)):
  predictions.append(globalAverage)

validation_ratings = [d['rating'] for d in rating_validation]

# Get performance on the validation set
performance(predictions, validation_ratings)

Global average: 4.18703


0.7483437444993984

In [0]:
### Training predictor

# Get all unique users and businesses in training set
temp = set(d['userID'] for d in rating_training)
unique_userID = {}
for val in temp:
  unique_userID[val] = []
temp = set(d['businessID'] for d in rating_training)
unique_businessID = {}
for val in temp:
  unique_businessID[val] = []

In [0]:
# Calculates the predicted rating

def calculate(myalpha, beta_u, beta_i, myuserid, myitemid):
  returnval = myalpha
  if myuserid in beta_u:
    returnval += beta_u[myuserid]
  if myitemid in beta_i:
    returnval += beta_i[myitemid]
  return returnval

In [0]:
### Training starts here

# Init alpha to the global average and betas to random values
alpha = globalAverage
beta_user = {}
beta_item = {}
for data in unique_userID:
  beta_user[data] = random.uniform(0, 1)
for data in unique_businessID:
  beta_item[data] = random.uniform(0, 1)

# Calculate initial performance
predictions = [calculate(alpha, beta_user, beta_item, d['userID'], d['businessID']) for d in rating_training]
training_ratings = [d['rating'] for d in rating_training]

myperform = performance(predictions, training_ratings)
# Adding 1 here so loop is guaranteed to start
myperform_old = myperform + 1

# Defining lambda here
lam = 1

# Stop updating when performance doesn't increase by at least delta
delta = 0.000001

while (myperform_old - myperform >= delta):
  # Define some intermediary data structures to speed up loop
  beta_user_info = {}
  for u in beta_user:
    beta_user_info[u] = [0,0]
  beta_item_info = {}
  for i in beta_item:
    beta_item_info[i] = [0,0]
  
  # Update alpha
  temp = 0
  for data in rating_training:
    temp += (data['rating'] - (beta_user[data['userID']] + beta_item[data['businessID']]))
  alpha = temp / len(rating_training)
  
  # Update beta_user
  for data in rating_training:
    beta_user_info[data['userID']][0] += (data['rating'] - (alpha + beta_item[data['businessID']]))
    beta_user_info[data['userID']][1] += 1
  for u in beta_user_info:
    beta_user[u] = beta_user_info[u][0] / (lam + beta_user_info[u][1])
  
  # Update beta_item
  for data in rating_training:
    beta_item_info[data['businessID']][0] += (data['rating'] - (alpha + beta_user[data['userID']]))
    beta_item_info[data['businessID']][1] += 1
  for i in beta_item_info:
    beta_item[i] = beta_item_info[i][0] / (lam + beta_item_info[i][1])
  
  # Save old performance
  myperform_old = myperform
  # Calculate new performance
  predictions = [calculate(alpha, beta_user, beta_item, d['userID'], d['businessID']) for d in rating_training]
  myperform = performance(predictions, training_ratings)
  # Display new performance
  print ("MSE: " + str(myperform))
  #print ("alpha: " + str(alpha))
  #print ("beta_user: ")
  #print list(beta_user.values())[:5]
  #print ("beta_item: ")
  #print list(beta_item.values())[:5]
  print ("Improvement: " + str(myperform_old - myperform))

MSE: 0.365899971999
Improvement: 1.53823132335
MSE: 0.344309914154
Improvement: 0.0215900578458
MSE: 0.340944854987
Improvement: 0.00336505916637
MSE: 0.339215944508
Improvement: 0.00172891047909
MSE: 0.337861389304
Improvement: 0.00135455520472
MSE: 0.336686965262
Improvement: 0.00117442404124
MSE: 0.335652526433
Improvement: 0.00103443882909
MSE: 0.334746489898
Improvement: 0.000906036535282
MSE: 0.333961229404
Improvement: 0.000785260493532
MSE: 0.333288031618
Improvement: 0.000673197786794
MSE: 0.332716686802
Improvement: 0.000571344815405
MSE: 0.332236126236
Improvement: 0.000480560565806
MSE: 0.331835137806
Improvement: 0.000400988430619
MSE: 0.331502916377
Improvement: 0.000332221428712
MSE: 0.33122942112
Improvement: 0.000273495257025
MSE: 0.331005572089
Improvement: 0.000223849031162
MSE: 0.330823328771
Improvement: 0.00018224331741
MSE: 0.330675688499
Improvement: 0.000147640272765
MSE: 0.330556634285
Improvement: 0.000119054213264
MSE: 0.330461053712
Improvement: 9.558057297

In [0]:
# Getting the MSE on the validation set
predictions = [calculate(alpha, beta_user, beta_item, d['userID'], d['businessID']) for d in rating_validation]
validation_ratings = [d['rating'] for d in rating_validation]

myperform = performance(predictions, validation_ratings)
print ("MSE on the validation set: " + str(myperform))

MSE on the validation set: 0.645683469177


In [0]:
### Finding the user and item IDs that have the largest and smallest beta values
largest_beta_user = ['temp', 0]
smallest_beta_user = ['temp', 0]
for u in beta_user:
  if beta_user[u] > largest_beta_user[1]:
    largest_beta_user = [u, beta_user[u]]
  if beta_user[u] < smallest_beta_user[1]:
    smallest_beta_user = [u, beta_user[u]]

largest_beta_item = ['temp', 0]
smallest_beta_item = ['temp', 0]
for i in beta_item:
  if beta_item[i] > largest_beta_item[1]:
    largest_beta_item = [i, beta_item[i]]
  if beta_item[i] < smallest_beta_item[1]:
    smallest_beta_item = [i, beta_item[i]]

# Display results
print ("The userID with the largest value of beta: " + largest_beta_user[0] + " with a beta of " + str(largest_beta_user[1]))
print ("The userID with the smallest value of beta: " + smallest_beta_user[0] + " with a beta of " + str(smallest_beta_user[1]))
print ("The businessID with the largest value of beta: " + largest_beta_item[0] + " with a beta of " + str(largest_beta_item[1]))
print ("The businessID with the smallest value of beta: " + smallest_beta_item[0] + " with a beta of " + str(smallest_beta_item[1]))

The userID with the largest value of beta: U357799541 with a beta of 1.16292548699
The userID with the smallest value of beta: U417838537 with a beta of -2.83465156955
The businessID with the largest value of beta: B093985406 with a beta of 1.17443708233
The businessID with the smallest value of beta: B241777680 with a beta of -2.22832787751


In [0]:
### Finding a better lambda

# Init betas to random values and freeze for better comparison
beta_user_init = {}
beta_item_init = {}
for data in unique_userID:
  beta_user_init[data] = random.uniform(0, 1)
for data in unique_businessID:
  beta_item_init[data] = random.uniform(0, 1)

# Defining pipeline here
def mypipeline(lam):
  # Init alpha
  alpha = globalAverage
  # Grab the frozen values
  beta_user = {}
  for data in beta_user_init:
    beta_user[data] = beta_user_init[data]
  beta_item = {}
  for data in beta_item_init:
    beta_item[data] = beta_item_init[data]

  # Calculate initial performance
  predictions = [calculate(alpha, beta_user, beta_item, d['userID'], d['businessID']) for d in rating_training]
  training_ratings = [d['rating'] for d in rating_training]

  myperform = performance(predictions, training_ratings)
  # Adding 1 here so loop is guaranteed to start
  myperform_old = myperform + 1

  # Stop updating when performance doesn't increase by at least delta
  delta = 0.000001

  while (myperform_old - myperform >= delta):
    # Define some intermediary data structures to speed up loop
    beta_user_info = {}
    for u in beta_user:
      beta_user_info[u] = [0,0]
    beta_item_info = {}
    for i in beta_item:
      beta_item_info[i] = [0,0]
  
    # Update alpha
    temp = 0
    for data in rating_training:
      temp += (data['rating'] - (beta_user[data['userID']] + beta_item[data['businessID']]))
    alpha = temp / len(rating_training)
  
    # Update beta_user
    for data in rating_training:
      beta_user_info[data['userID']][0] += (data['rating'] - (alpha + beta_item[data['businessID']]))
      beta_user_info[data['userID']][1] += 1
    for u in beta_user_info:
      beta_user[u] = beta_user_info[u][0] / (lam + beta_user_info[u][1])
  
    # Update beta_item
    for data in rating_training:
      beta_item_info[data['businessID']][0] += (data['rating'] - (alpha + beta_user[data['userID']]))
      beta_item_info[data['businessID']][1] += 1
    for i in beta_item_info:
      beta_item[i] = beta_item_info[i][0] / (lam + beta_item_info[i][1])
  
    # Save old performance
    myperform_old = myperform
    # Calculate new performance
    predictions = [calculate(alpha, beta_user, beta_item, d['userID'], d['businessID']) for d in rating_training]
    myperform = performance(predictions, training_ratings)

  # Evaluate on validation set
  predictions = [calculate(alpha, beta_user, beta_item, d['userID'], d['businessID']) for d in rating_validation]
  validation_ratings = [d['rating'] for d in rating_validation]

  myperform = performance(predictions, validation_ratings)
  return [alpha, beta_user, beta_item, myperform]

mylambdas = [0.01, 0.1, 1, 10, 100, 1000]
bestalpha = 0
bestbeta_user = {}
bestbeta_item = {}

for mylambda in mylambdas:
  myresults = mypipeline(mylambda)
  mse = myresults[3]
  print ("MSE for lambda = " + str(mylambda) + " is: " + str(mse))
  if mylambda == 10:
    bestalpha = myresults[0]
    for data in myresults[1]:
      bestbeta_user[data] = myresults[1][data]
    for data in myresults[2]:
      bestbeta_item[data] = myresults[2][data]

MSE for lambda = 0.01 is: 0.836895749419
MSE for lambda = 0.1 is: 0.77657881585
MSE for lambda = 1 is: 0.645683507635
MSE for lambda = 10 is: 0.621424407266
MSE for lambda = 100 is: 0.704180633989
MSE for lambda = 1000 is: 0.741562727755


In [0]:
### Running on test data

predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  predictedRating = calculate(bestalpha, bestbeta_user, bestbeta_item, u, i)
  predictions.write(u + '-' + i + "," + str(predictedRating) + "\n")

files.download('predictions_Rating.txt')